In [113]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [114]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

In [115]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split 
import xgboost as xgb
import seaborn as sns
import os
import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

# does not include dielectric constant and molar volume. 
# querying script to be updated when doable

In [116]:
os.chdir('/gdrive/My Drive/Datasets/')

In [117]:
# read data frame overall
df = pd.read_csv("aqsol.csv")

In [118]:
# df.columns
df.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,MolLogP,MolMR,HeavyAtomCount,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,3.9581,102.4454,23.0,0.0,0.0,2.0,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,2.4055,51.9012,13.0,1.0,1.0,2.0,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,2.1525,36.8395,9.0,1.0,0.0,2.0,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,8.1161,200.7106,53.0,6.0,2.0,7.0,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,2.4854,119.0760,31.0,6.0,0.0,6.0,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


In [119]:
# X - NumHAcceptors, NumHDonors, NumAromaticRings, NumSaturatedRings, NumAliphaticRings, MolMR
# Y - Solubility 
# Merged dataframe to provide pair plot 

X = df.loc[:, ['MolMR','NumHAcceptors', 'NumHDonors', 'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings']]
Y = df.loc[:, ['Solubility']]
merge = df.loc[:, ['MolMR','NumHAcceptors', 'NumHDonors', 'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings', 'Solubility']]

In [120]:
# Perform pairwise plots to visualize data
#plot = sns.pairplot(merge)

In [121]:
# Performing linear regression with defined X and Y parameters
# not a very good R^2 of 0.39282.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
regressor = LinearRegression()

reg = regressor.fit(X_train, Y_train)

reg.score (X_train, Y_train)

0.39282680008336895

In [122]:
xgb = xgb.XGBRegressor()

parameters = {'nthread':[4],
              'objective':['reg:linear'],
              'learning_rate': [0.005, 0.01, 0.03, 0.05, .07],
              'max_depth': [5, 6, 7],
              'min_child_weight': [3, 4, 5],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7, 0.9, 1.0],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb,
                        parameters,
                        cv = 2,
                        n_jobs = -1,
                        verbose=True)
xgb_grid.fit(X_train, Y_train)

print('XGBoost Regressor Score is {}'.format(xgb_grid.score(X_train, Y_train)))

Fitting 2 folds for each of 135 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  5.8min finished


XGBoost Regressor Score is 0.7295869757585601


In [123]:
xgb_grid.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.01,
 'max_depth': 7,
 'min_child_weight': 3,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:linear',
 'silent': 1,
 'subsample': 0.7}

In [124]:
print('Linear Regression R2 Score is {} %\nXGBoost Regressor with basic GridSearchCV R2 Score is {} %'.format(reg.score(X_train, Y_train) * 100, xgb_grid.score(X_train, Y_train) * 100))
print('')
print('As you can see, this is a big leap in accuracy.')

Linear Regression R2 Score is 39.2826800083369 %
XGBoost Regressor with basic GridSearchCV R2 Score is 72.95869757585601 %

As you can see, this is a big leap in accuracy.


In [125]:
# showing the predicted solubility coefficients of the purely linear model 

temp = pd.DataFrame(regressor.coef_)
coeff_df = temp.T
coeff_df.index = ['MolMR','NumHAcceptors', 'NumHDonors', 'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings']
coeff_df.columns = ['Predicted Solubility']
coeff_df

,Predicted Solubility
MolMR,-0.030925
NumHAcceptors,0.267275
NumHDonors,0.321167
NumAromaticRings,-0.370889
NumSaturatedRings,-0.072357
NumAliphaticRings,-0.077015


In [126]:
# evaluating fit for training data

Y_pred = regressor.predict(X_train)

xgb_pred = xgb_grid.predict(X_train)

In [127]:
# more information about MAE, MSE and RMSE 

print('Linear Regression Mean Absolute Error:', metrics.mean_absolute_error(Y_train, Y_pred))  
print('Linear Regression Mean Squared Error:', metrics.mean_squared_error(Y_train, Y_pred))  
print('Linear Regression Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_train, Y_pred)))
print('Mean:', Y_train.mean())
print('Linear Regression Mean:', Y_pred.mean())

Linear Regression Mean Absolute Error: 1.386096546511378
Linear Regression Mean Squared Error: 3.4187900469999817
Linear Regression Root Mean Squared Error: 1.848997038126341
Mean: Solubility   -2.893181
dtype: float64
Linear Regression Mean: -2.8931812180989978


In [128]:
print('Linear Regression Mean Absolute Error:', metrics.mean_absolute_error(Y_train, xgb_pred))  
print('Linear Regression Mean Squared Error:', metrics.mean_squared_error(Y_train, xgb_pred))  
print('Linear Regression Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_train, xgb_pred)))
print('Mean:', Y_train.mean())
print('XGBRegressor Mean:', xgb_pred.mean())

Linear Regression Mean Absolute Error: 0.895892912212555
Linear Regression Mean Squared Error: 1.5226056683376956
Linear Regression Root Mean Squared Error: 1.2339390861536463
Mean: Solubility   -2.893181
dtype: float64
XGBRegressor Mean: -2.8681843


In [131]:
yt_pred = regressor.predict(X_test)
xgbt_pred = xgb_grid.predict(X_test)

In [133]:
yt_score = regressor.score(X_test, Y_test)
xgbt_score = xgb_grid.score(X_test, Y_test)

In [136]:
print("Test Score of Linear Regression is {} %\nTest Score of XGBRegressor is {} %".format(yt_score * 100, xgbt_score * 100))

Test Score of Linear Regression is 31.024951272218438 %
Test Score of XGBRegressor is 61.600429589616134 %


In [129]:
# this isn't really an overfitting problem. 
# It's more related to the lack of dielectric constant + the other variable